# Regularization technique like Lasso can be used as a variable selection method

In [2]:
import graphlab
import pandas as pd
import numpy as np

In [3]:
# Load data fiels
vehicle = pd.read_csv('vehicles.csv')
reservation = pd.read_csv('reservations.csv')

In [4]:
# Roll up the reservation data and merge it with the vehicle data
reserv_agg = reservation.groupby('vehicle_id')
reserv_agg = pd.DataFrame(reserv_agg.size().reset_index(name="count"))
merge = vehicle.merge(reserv_agg,on='vehicle_id', how='left')
merge['count'].fillna(0, inplace=True) 

In [5]:
# Change the data frame into graphlab Sframe, as we have direct libraries to implement Lasso on Sframe dataset
merge_s = graphlab.SFrame(merge)

This non-commercial license of GraphLab Create for academic use is assigned to sdas26@uic.edu and will expire on September 26, 2017.


[INFO] graphlab.cython.cy_server: GraphLab Create v2.1 started. Logging: C:\Users\Sanjib\AppData\Local\Temp\graphlab_server_1489986391.log.0


In [11]:
merge_s.head()

vehicle_id,technology,actual_price,recommended_price,num_images,street_parked,description,count
1,1,67.85,59,5,0,7,1.0
2,0,100.7,53,5,0,224,7.0
3,0,74.0,74,4,1,21,17.0
4,0,135.0,75,1,0,184,2.0
5,0,59.36,53,2,1,31,2.0
6,0,82.15,53,5,1,44,13.0
7,0,165.17,83,3,0,32,0.0
8,0,98.56,88,5,1,174,8.0
9,1,69.96,44,4,1,32,11.0
10,0,98.04,76,4,1,204,9.0


In [6]:
# List all the features to be used to train the model
all_features = ['technology', 'actual_price','recommended_price','num_images', 'street_parked','description']

In [18]:
# Train a linear regression model with lasso penalty value
model_all = graphlab.linear_regression.create(merge_s, target='count', features=all_features,
                                              validation_set=None, 
                                              l2_penalty=0., l1_penalty=0.01, verbose= False)

In [19]:
coeff = model_all["coefficients"][("name", "value")]

In [20]:
# Get the Coefficients for each feature
coeff.sort('value', ascending=False)

name,value
technology,2.52765394249
(intercept),1.73141151433
num_images,0.67905524029
street_parked,0.244007190808
recommended_price,0.0233616399879
description,0.00359172540706
actual_price,0.00179237038391
